In [8]:
from __future__ import print_function

import argparse
import os
import time

import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim

from helpers.loaders import *
from helpers.utils import adjust_learning_rate
from models import *
from trainer2_0 import test, train

In [3]:
lr=0.1
train_db_path='./data'
test_db_path = './data'
dataset = 'cifar10'

batch_size = 128 # 本来为128
wm_batch_size = 2
max_epochs = 80 #从头开始训练的epoch为80
lradj = 20
save_dir='./saved/'
#save_model='save_cifar10_resnet_53ed'
#load_path = './save_cifar10_resnet_53ed'

testdataset = True

In [4]:
resume =True #丛头开始训练
wmtrain = True
wm_path = './trigger_set/cifar10'
# wmeh_path = './trigger_set/cifar10 m=8_6 enhance'
wmtest_path = './trigger_set/cifar10 test'
wm_lbl = 'cifar10_labels.txt'
wmeh_lbl = 'cifar10_labels.txt'
wmtest_lbl = 'cifar10_labels.txt'
log_dir = './log_cifar10_1009_3'
runname = 'train'

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

LOG_DIR = log_dir
if not os.path.isdir(LOG_DIR):
    os.mkdir(LOG_DIR)
logfile = os.path.join(LOG_DIR, 'log_' + str(runname) + '.txt')
confgfile = os.path.join(LOG_DIR, 'conf_' + str(runname) + '.txt')

# save configuration parameters
#with open(confgfile, 'w') as f:
#    for arg in vars(args):
#        f.write('{}: {}\n'.format(arg, getattr(args, arg)))

trainloader, testloader, n_classes = getdataloader(
   dataset, train_db_path, test_db_path, batch_size)

wmloader = None
wmtestloader = None
if wmtrain:
    print('Loading watermark images')
    wmloader = getwmloader(dataset,wm_path, batch_size, wm_lbl)
#   wmehloader = getwmloader(dataset,wmeh_path, batch_size, wmeh_lbl)
    wmtestloader = getwmtestloader(dataset,wmtest_path, wm_batch_size, wmtest_lbl)

==> Preparing data..
Using CIFAR10 dataset.
Files already downloaded and verified
Files already downloaded and verified
Loading watermark images


In [ ]:
#本地数据集分装的loader
trainloader=getwmloader(dataset,train_db_path, 32, wm_lbl)
testloader=getwmloader(dataset,test_db_path, 2, wm_lbl)

In [ ]:
#print(n_classes)
n_classes=10

In [ ]:
import matplotlib.pyplot as plt
to_pil_image = transforms.ToPILImage()
cnt = 0
for image,label in wmloader:
    if cnt>=3: # 只显示3张图片
        break
    print(label) # 显示label
 
 # 方法1：Image.show()
 # transforms.ToPILImage()中有一句
 # npimg = np.transpose(pic.numpy(), (1, 2, 0))
 # 因此pic只能是3-D Tensor，所以要用image[0]消去batch那一维
#img = to_pil_image(image[0])
#img.show()
 
 # 方法2：plt.imshow(ndarray)
    img = image[0] # plt.imshow()只能接受3-D Tensor，所以也要用image[0]消去batch那一维
    img = img.numpy() # FloatTensor转为ndarray
    img = np.transpose(img, (1,2,0)) # 把channel那一维放到最后
#     print(img)
    # 显示图片
    plt.imshow(img)
    plt.show()

    cnt += 1

In [6]:
# create the model
if resume ==False:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.exists("./saved/save_cifar10_resnet_53ed_0928"), 'Error: no checkpoint found!'
    checkpoint = torch.load("./saved/save_cifar10_resnet_53ed_0928")
    net = checkpoint['net']
    acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']
else:
    print('==> Building model..')
    net = ResNet18(num_classes=n_classes)
#   net = VGG('VGG19')

net = net.to(device)
# support cuda
if device == 'cuda':
    print('Using CUDA')
    print('Parallel training on {0} GPUs.'.format(torch.cuda.device_count()))
    net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)


==> Building model..
Using CUDA
Parallel training on 1 GPUs.


In [ ]:
# loading test examples
if testdataset :
    print("test acc:")
    test(net, criterion, logfile, testloader, device)

In [ ]:
# loading wm examples
if wmtrain:
    print("WM acc:")
    test(net, criterion, logfile, testloader, device)

In [9]:
# start training
save_dir='./saved/'
save_model='save_cifar10_resnet_53ed_1009_3'
for epoch in range(start_epoch, start_epoch + 10):
    # adjust learning rate
    #if wmtrain : # 从头开始训练就不用调整学习率了
    #   adjust_learning_rate(lr, optimizer, epoch,lradj)#预训练模型不用加微调学习率

    train(epoch, net, criterion, optimizer, logfile,
          trainloader, device, wmloader) #, wmehloader)

    print("Test acc:")
    acc = test(net, criterion, logfile, wmtestloader, device)

    #if wmtrain:
    #   print("WM acc:")
    #   test(net, criterion, logfile, wmloader, device)

    print('Saving..')
    state = {
        'net': net.module if device == 'cuda' else net,
        'acc': acc,
        'epoch': epoch,
    }
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
    torch.save(state, os.path.join(save_dir, save_model))



Epoch: 0


KeyboardInterrupt: 